## Flujo de un modelo de machine learning

<img src="https://ml-ops.org/img/ml-engineering.jpg"></img>

https://ml-ops.org/content/end-to-end-ml-workflow#model-deployment

## Requerimiento para hacer MLOps

https://ml-ops.org/content/three-levels-of-ml-software

## Arquitectura del proyecto

### ¿En qué consiste el proyecto?

    * Predecir la cantidad de dinero que genera una película a través de una API
    * Reproducir todo el flujo de puesta en producción de un modelo
    * Aplica entrenamiento continuo
    * Desplegar continuamente usando CI/CD

<img src="https://static.platzi.com/media/user_upload/Arquitecture-b82a207a-4d89-49af-988c-d0b410fe676c.jpg"></img>

https://ml-ops.org/content/three-levels-of-ml-software

## Distribución de archivos

* dvc/ <- configuraciones de DVC

* .github/workflows/ <- actions que se ejecutaran

* api/ <- API utilizando fast API

* datset/ <- archivos del dataset traqueados

* model/ <- archivos de modelos traqueados

* notebooks/ <- notebooks con el modelo

* src/ <- archivos usados para reentrenamiento

* utilities/ <- archivos con utilidades especificas

* … <- archivos misceláneos del proyecto y extras que se necesitaran

## Data Version Control DVC

https://dvc.org/doc

## Comandos de DVC

https://dvc.org/doc/command-reference

### Comandos básico
* dvc init
* dvc remote
* dvc add
* dvc pull
* dvc push
* dvc run
* dvc repro
* dvc dag


## implementar DVC en nuestro proyecto

* En google clous crear un cuenta y una llave de servicio

* en el ambiente se instala la libreria de dvc
    * conda install -c conda-forge mamba # installs much faster than conda
    * mamba install -c conda-forge dvc
    * mamba install -c conda-forge dvc-gs

* mamba install -c conda-forge dvc[gs]

* !dvc init

export GOOGLE_APPLICATION_CREDENTIALS=$(realpath .\mediapp-321819-7781fa950444.json)

* Crea en google cloud un bucket con dos carpetas para que guarde la información del dataset y del modelo

dvc remote ad
d dataset-track gs://model-dataset-tracker-caleb/dataset

dvc add dataset\finantials.csv --to-remote -r dataset-track

dvc add .\dataset\opening_gross.csv --to-remote -r dataset-track

## mejor versión
* dvc add dataset/finantials.csv
* dvc add dataset/opening_gross.csv
* dvc add dataset/movies.csv

* dvc add model/model.pkl

### Pushear al remoto

* dvc push dataset/finantials.csv -r dataset-track
* dvc push dataset/opening_gross.csv -r dataset-track
* dvc push dataset/movies.csv -r dataset-track

* dvc push model/model.pkl -r model-track

ls dataset/

dvc remote add model-tracker gs://model-dataset-tracker-caleb/model

dvc add model/model.pkl --to-remote -r model-tracker

ls model/

* crear en un archivo gitignore

venv/
-ipynb_checkpoints/
*.json
*.csv
*.pkl


## Desarrollo de pipeline de reentrenamiento: preparación de la data

git checkout -b continuous_training_pipeline

* se crea una carpeta src
* dentro de ella se crea los archivo: 
  * prepare.py
  * train.py
  * utils.py
  * requirements.txt

pip3 install -r ./src/requirements.txt

* python .\src\prepare.py para probar que la función correo bien

## Desarrollo de pipeline de reentrenamiento: script de entrenamiento

Escribir el código del archivo train.py

## Desarrollo de pipeline de reentrenamiento: validación del modelo

Despues de generar los archivos train y utils se corre el siguiente codigo en consola

* dvc run -> crear un paso para guardar un script para preparar la data

dvc run -n prepare -o dataset/full_data.csv python src/prepare.py

* dvc run -> crear un paso para guardar un script para entrenar el modelo 

dvc run -n traning -d dataset/full_data.csv python src/train.py

dvc repro -> para reproducir estos script esto funciona cuando alguien hace un push a los datos o se actualizan los datos o modelo

dvc repro -f -> fuerza la reproducción del proceso

dvc dag -> para ver los dags que se están formando

## Desarrollo de API con FastAPI

Se crea la carpeta api y dentro el archivo main.py y requirements.txt

Luego se crea la carpeta app y dentro los archivos models.py, views.py y utils.py para crear el api

En el archivo main.py se crea la rutal del api

para correr el servidor uvicorn api.main:app

objeto de pruebas en la api:

{
  "opening_gross": 8330681,
  "screens": 2271,
  "production_budget": 13000000,
  "title_year": 1999,
  "aspect_ratio": 1.85,
  "duration": 97,
  "cast_total_facebook_likes": 37907,
  "budget": 16000000,
  "imdb_score": 7.2
}

## Empaquetando API con Docker

Se crea el archivo Dockerfile con la configuración del docker y el archivo initializer.sh para uniciar el servidor

docker build . -t model-api:v1

docker run -p 8000:8000 model-api:v1

* docker ps -> para saber el estado
* docker stop 47aa4fc60fff -> para detener el contenedor con el id de la imagen

## Presentacion de Github Actions y Continuous Machine Learning

* https://docs.github.com/en/actions
* https://github.com/iterative/cml

## Desarrollo de workflow para testing

* Se crea una carpeta .github
* se crea la carpeta workflow y dentro el archivo de configuración .yaml, llamado testing.yalm con la configuración necesaria
*  Se crea una carpeta llamada utilities que contendra el archivo que genera el archivo encriptado de la secret key de google
*  ir a gcp y en APIs y servicio en la opción de credenciales
*  Sea crea una nueva credencial de service accounts con los permisos de service 
*  en la consola de ubuntu se escribe base64 y el nombre del archivo para que genere el b64 y ese se copia en github para crear el testin
*  en github se va settings y en secret actions se crea la llave con el nombre de SERVICE_ACCOUNT_KEY y el b64 copiado